In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using LinearAlgebra
using CairoMakie

In [ ]:
function transmon(n::Int)
    at = create(n)
    a = annihilate(n)
    H_0 = zeros(n, n)
    H_C = [at + a, im * (at - a)]
    return H_0, H_C
end

In [ ]:
probs = Dict()

In [ ]:
hsd = 2
H_drift, H_drives = transmon(hsd)
X_goal = embed_operator(hsd, GATES[:X])
SX_goal = embed_operator(hsd, sqrt(GATES[:X]))
subspace = subspace_indices([hsd])
T = 50
Δt = .2

# X gate
probs["X"] = UnitarySmoothPulseProblem(
    H_drift, H_drives, X_goal, T, Δt;
    geodesic=true,
    subspace=subspace,
    verbose=false,
    timesteps_all_equal=true,
    free_time=false,
    hessian_approximation=true,
    pade_order=10,
    R=1.,
)

solve!(probs["X"]; max_iter=200)

# SX gate
probs["SX"] = UnitarySmoothPulseProblem(
    H_drift, H_drives, SX_goal, T, Δt;
    geodesic=true,
    subspace=subspace,
    verbose=false,
    timesteps_all_equal=true,
    free_time=false,
    hessian_approximation=true,
    pade_order=10,
    R=1.,
)

solve!(probs["SX"]; max_iter=200)

In [ ]:
unitary_fidelity(probs["X"]) |> println
unitary_fidelity(probs["SX"]) |> println

# Combine

In [ ]:
B_drives = Matrix{ComplexF64}[]
append!(B_drives, [H ⊗ GATES[:I] for H in H_drives])
append!(B_drives, [GATES[:I] ⊗ H  for H in H_drives])

B_guess = []
for _ in 1:2
    for row ∈ eachrow(probs["X"].trajectory[:a])
        push!(B_guess, copy(row))
    end
end
B_guess = collect(hcat(B_guess...)')

B_goal = X_goal ⊗ X_goal

ZZ = GATES[:Z] ⊗ GATES[:Z]

In [ ]:
two_subspace = subspace_indices([hsd, hsd])
probs["pair"] = UnitarySmoothPulseProblem(
    zeros(4, 4), B_drives, B_goal, T, Δt;
    # a_guess=B_guess,
    subspace=two_subspace,
    timesteps_all_equal=false,
    free_time=true,
    hessian_approximation=true,
    pade_order=10,
    R=1.,
)

In [ ]:
solve!(probs["pair"]; max_iter=200)

In [ ]:
unitary_fidelity(probs["pair"]) |> println

In [ ]:
plot(probs["pair"].trajectory)

## Robust

In [ ]:
probs["robust"] = UnitaryRobustnessProblem(
    ZZ, probs["pair"];
    final_fidelity=0.99, subspace=two_subspace, verbose=false,
    hessian_approximation=true
)

solve!(probs["robust"]; max_iter=100)

In [ ]:
plot(probs["robust"].trajectory)

In [ ]:
unitary_fidelity(probs["robust"]) |> println

In [ ]:
# --------------------------------------------
# 4. test UnitaryRobustnessProblem default
# --------------------------------------------
params = deepcopy(probs["pair"].params)
trajectory = copy(probs["pair"].trajectory)
system = probs["pair"].system
objective = QuadraticRegularizer(:dda, trajectory, 1e-4)
objective += QuadraticRegularizer(:a, trajectory, 1e-4)
integrators = probs["pair"].integrators
constraints = AbstractConstraint[]

probs["unconstrained"] = UnitaryRobustnessProblem(
    ZZ, trajectory, system, objective, integrators, constraints;
    final_fidelity=0.9999, subspace=two_subspace, verbose=false,
    hessian_approximation=true
)

solve!(probs["unconstrained"]; max_iter=100)

In [ ]:
unitary_fidelity(probs["unconstrained"])

In [ ]:
loss = (Hₑ, prob) -> begin
    Z⃗ = vec(prob.trajectory.data)
    Z = prob.trajectory
    return InfidelityRobustnessObjective(Hₑ, Z).L(Z⃗, Z)
end

In [ ]:
loss(ZZ, probs["unconstrained"])

In [ ]:
loss(ZZ, probs["pair"])


In [ ]:
plot(probs["unconstrained"].trajectory)

In [ ]:
plot(probs["unconstrained"].trajectory)
